# <font color='LightBlue'> **Transportation Model**</font>

<font color='Silver'>
Kevin Schaaf ()
Haron Malash ()
Sanjhana Bhusal ()
Martijn Messanella ()
Lisa Pahladsingh ()
</font>

In [19]:
from Simulate import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import pandas as pd

plt.style.use('default')

## <font color='LightGreen'> ***Capacity***</font>
Kevin

In [20]:
def vary_capacity_experiment(num_commuters, islands, capacity, metrics_to_plot, ferry_base_price):
    '''
    This function executes the capacity experiment. 
    args:
        num_commuters:   The number of agents in the model
        islands:         The names of the islands included in the model
        capacities:      A list or array of integers, with the lowest integer the number of agents
        metrics_to_plot: A list of all the metrics you want to plot; 
                         This should be a list of strings with the names of the metrics.
                         Also, this variable might need adjustment for different number of islands
    Output:
        Plot of the percentage of ferry users for this settings of the experiment. 
    '''
    simulation = Simulation(
        num_commuters=num_commuters,
        num_days=100,
        islands=islands,
        capacity=capacity,
        ferry_base_price=ferry_base_price,
        ferry_base_time=40,
        speedboat_base_price=6,
        speedboat_base_time=10,
    )

    simulation.run()

    # Plot specific results
    metrics_to_plot = [
    'Ferry_Island_A_Island_B_users', 'Ferry_Island_A_Island_C_users', 'Ferry_Island_B_Island_C_users',
    'Ferry_Island_B_Island_A_users', 'Ferry_Island_C_Island_A_users', 'Ferry_Island_C_Island_B_users',
    'Speedboat_Island_A_Island_B_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_B_Island_C_users',
    'Speedboat_Island_B_Island_A_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_C_Island_B_users'
    ]

    return simulation.return_percentage_ferry_users(metrics_to_plot)

In [21]:
islands = ["Island_A", "Island_B", "Island_C"]
num_commuters = 1000
prices = np.arange(1, 11, 1)
capacities = np.arange(200, 2200, 200)
runs = 15
ferry_base_price = 1

num_days = 100
metrics_to_plot = [
    'Ferry_Island_A_Island_B_users', 'Ferry_Island_A_Island_C_users', 'Ferry_Island_B_Island_C_users',
    'Ferry_Island_B_Island_A_users', 'Ferry_Island_C_Island_A_users', 'Ferry_Island_C_Island_B_users',
    'Speedboat_Island_A_Island_B_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_B_Island_C_users',
    'Speedboat_Island_B_Island_A_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_C_Island_B_users'
    ]

percentages_ferry_users = np.zeros((len(capacities), len(prices), num_days, runs))
mean = np.zeros((len(capacities), len(prices), num_days))
CI = np.zeros((len(capacities), len(prices), num_days, 2))

mean_mean = np.zeros((len(capacities), len(prices)))
CI_mean = np.zeros((len(capacities), len(prices), 2))
capacity = 1000

for price_ix, ferry_base_price in enumerate(prices):
    for capacity_ix, capacity in enumerate(capacities):
        for i in range(runs):
            percentages_ferry_users[capacity_ix, price_ix, :, i] = vary_capacity_experiment(num_commuters, islands, capacity, metrics_to_plot, ferry_base_price)

        mean[capacity_ix, price_ix] = np.mean(percentages_ferry_users[capacity_ix, price_ix], axis=1)
        CI[capacity_ix, price_ix] = np.percentile(percentages_ferry_users[capacity_ix, price_ix], [2.5, 97.5], axis=1).T

        mean_mean[capacity_ix, price_ix] = np.mean(mean[capacity_ix, price_ix, -10:])
        CI_mean[capacity_ix, price_ix] = np.percentile(mean[capacity_ix, price_ix, -10:], [2.5, 97.5])


In [22]:
df_percentages = pd.DataFrame(percentages_ferry_users.reshape(len(capacities)*len(prices)*num_days*runs))
df_mean = pd.DataFrame(mean.reshape(len(capacities)*len(prices)*num_days))
df_CI = pd.DataFrame(CI.reshape(len(capacities)*len(prices)*num_days*2))
df_mean_mean = pd.DataFrame(mean_mean.reshape(len(capacities)*len(prices)))
df_CI_mean = pd.DataFrame(CI_mean.reshape(len(capacities)*len(prices)*2))

# Save DataFrames to CSV
df_percentages.to_csv('Data3/percentages_ferry_users.csv', index=False)
df_mean.to_csv('Data3/mean.csv', index=False)
df_CI.to_csv('Data3/CI.csv', index=False)
df_mean_mean.to_csv('Data3/mean_mean.csv', index=False)
df_CI_mean.to_csv('Data3/CI_mean.csv', index=False)

In [37]:
def save_Back_up_Data():
    df_percentages.to_csv('Back_up_Data3/percentages_ferry_users.csv', index=False)
    df_mean.to_csv('Back_up_Data3/mean.csv', index=False)
    df_CI.to_csv('Back_up_Data3/CI.csv', index=False)
    df_mean_mean.to_csv('Back_up_Data3/mean_mean.csv', index=False)
    df_CI_mean.to_csv('Back_up_Data3/CI_mean.csv', index=False)

# # If the run was a quality run, uncomment the next line 
save_Back_up_Data()

In [24]:
df_percentages = pd.read_csv('Data3/percentages_ferry_users.csv')
df_mean = pd.read_csv('Data3/mean.csv')
df_CI = pd.read_csv('Data3/CI.csv')
df_mean_mean = pd.read_csv('Data3/mean_mean.csv')
df_CI_mean = pd.read_csv('Data3/CI_mean.csv')

# Get the original shapes
num_days = 100
num_prices = len(prices)
num_capacities = len(capacities)
runs = 15

# Reshape the data to their original shapes
percentages_ferry_users = df_percentages.values.reshape(num_capacities, num_prices, num_days, runs)
mean = df_mean.values.reshape(num_capacities, num_prices, num_days)
CI = df_CI.values.reshape(num_capacities, num_prices, num_days, 2)
mean_mean = df_mean_mean.values.reshape(num_capacities, num_prices)
CI_mean = df_CI_mean.values.reshape(num_capacities, num_prices, 2)

In [25]:
for price_ix, ferry_base_price in enumerate(prices):
    plt.figure(dpi=300)
    for capacity_ix, capacity in enumerate(capacities):
        plt.plot(mean[capacity_ix, price_ix]*100, label=capacity)
        plt.fill_between(np.arange(0, num_days, 1), CI[capacity_ix, price_ix, :, 0]*100, CI[capacity_ix, price_ix, :, 1]*100, alpha=0.5)
    plt.title(f'Ferry Users for Price = {ferry_base_price}', fontsize=20, fontweight='bold')
    plt.legend()
    plt.xlabel('Time [a.u.]', fontsize=14, fontweight='bold')
    plt.ylabel('Ferry Users (%)', fontsize=14, fontweight='bold')
    plt.savefig(f'Figures3/Percentage of ferry users in time for price = {ferry_base_price}')
    plt.show()

In [26]:
for capacity_ix, capacity in enumerate(capacities):
    plt.figure(dpi=300)
    for price_ix, ferry_base_price in enumerate(prices):
        plt.plot(mean[capacity_ix, price_ix]*100, label=ferry_base_price)
        plt.fill_between(np.arange(0, num_days, 1), CI[capacity_ix, price_ix, :, 0]*100, CI[capacity_ix, price_ix, :, 1]*100, alpha=0.5)
    plt.title(f'Ferry Users for Capacity = {capacity}', fontsize=20, fontweight='bold')
    plt.legend()
    plt.xlabel('Time [a.u.]', fontsize=14, fontweight='bold')
    plt.ylabel('Ferry Users (%)', fontsize=14, fontweight='bold')
    plt.savefig(f'Figures3/Percentage of ferry users in time for capacity = {capacity}')
    plt.show()

C:\Users\kevin\AppData\Local\Temp\ipykernel_33968\3420966354.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(dpi=300)


In [27]:
plt.figure(dpi=300)
capacity_ix = 4
plt.plot(prices, mean_mean[capacity_ix, :]*100)
plt.fill_between(prices, CI_mean[capacity_ix, :, 0]*100, CI_mean[capacity_ix, :, 1]*100, alpha=0.5)
plt.title('Ferry users against price for capacity = 1000', fontsize=20, fontweight='bold')
plt.xlabel('Prices [a.u.]', fontsize=14, fontweight='bold')
plt.ylabel('Ferry Users (%)', fontsize=14, fontweight='bold')
plt.savefig('Figures3/Ferry users agains price for capacity = 1000')
plt.show()

In [28]:
plt.figure(dpi=300)
price_ix = 4
plt.plot(capacities, mean_mean[:, price_ix]*100)
plt.fill_between(capacities, CI_mean[:, price_ix, 0]*100, CI_mean[:, price_ix, 1]*100, alpha=0.5)
plt.title('Ferry users agains capacity for price = 5', fontsize=20, fontweight='bold')
plt.xlabel('Capacities', fontsize=14, fontweight='bold')
plt.ylabel('Ferry users (%)', fontsize=14, fontweight='bold')
plt.savefig('Figures3/Ferry users agains capacity for price = 5')
plt.show() 

In [29]:
%matplotlib

Using matplotlib backend: QtAgg


In [30]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(dpi=300)
ax = fig.add_subplot(111, projection='3d')

# Create meshgrid for capacities and prices
X, Y = np.meshgrid(capacities, prices)
Z = mean_mean.T  # Transpose to match the meshgrid dimensions

# Plot the surface
surf = ax.plot_surface(X, Y, Z*100, cmap='viridis', edgecolor='none')

# Add labels and title
ax.set_xlabel('Capacity', fontsize=14, fontweight='bold')
ax.set_ylabel('Price [a.u.]', fontsize=14, fontweight='bold')
ax.set_zlabel('Mean Ferry Users (%)', fontsize=14, fontweight='bold')
ax.set_title('Influence of Pricing and Capacity Policies on Ferry Usage ', fontsize=20, fontweight='bold')


# Add a color bar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

plt.savefig('Figures3/3D Surface Plot of Mean Percentage of Ferry Users')
plt.show()

In [52]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming mean_mean is already defined and contains the dependent variable values

# Calculate the partial derivatives
dZ_dX = np.gradient(mean_mean, axis=0)  # Partial derivative with respect to capacity
dZ_dY = np.gradient(mean_mean, axis=1)  # Partial derivative with respect to price

# Determine the effect at each point
effect = np.zeros_like(mean_mean)
effect[dZ_dX > dZ_dY] = 1  # Price has a greater effect
effect[dZ_dX <= dZ_dY] = 2  # Capacity has a greater effect

# Create meshgrid for capacities and prices
X, Y = np.meshgrid(capacities, prices)
Z = mean_mean.T  # Transpose to match the meshgrid dimensions

# Plot the surface
fig = plt.figure(dpi=300)
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, Z*100, cmap='viridis', edgecolor='none')

# Add labels and title
ax.set_xlabel('Capacity', fontsize=14, fontweight='bold')
ax.set_ylabel('Price [a.u.]', fontsize=14, fontweight='bold')
ax.set_zlabel('Mean Ferry Users (%)', fontsize=14, fontweight='bold')
ax.set_title('Influence of Pricing and Capacity Policies on Ferry Usage', fontsize=20, fontweight='bold')

# Add a color bar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

plt.savefig('Figures3/3D_Surface_Plot_of_Mean_Percentage_of_Ferry_Users.png')
plt.show()

# Plot the effect
fig, ax = plt.subplots(dpi=300)
c = ax.pcolormesh(X, Y, effect.T, cmap='coolwarm', shading='auto')
ax.set_xlabel('Capacity', fontsize=14, fontweight='bold')
ax.set_ylabel('Price [a.u.]', fontsize=14, fontweight='bold')
ax.set_title('Effect of Pricing vs Capacity on Ferry Usage', fontsize=20, fontweight='bold')
fig.colorbar(c, ax=ax, label='Effect (1: Price, 2: Capacity)')

plt.savefig('Figures3/Effect_of_Pricing_vs_Capacity_on_Ferry_Usage.png')
plt.show()

## <font color='LightGreen'> ***Number of Islands***</font>
Three Islands

Price


In [32]:
# islands = ["Island_A", "Island_B", "Island_C"]
# num_commuters = 1000

# fares = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# # Plot specific results
# metrics_to_plot = [
#     'Ferry_Island_A_Island_B_users', 'Ferry_Island_A_Island_C_users', 'Ferry_Island_B_Island_C_users',
#     'Ferry_Island_B_Island_A_users', 'Ferry_Island_C_Island_A_users', 'Ferry_Island_C_Island_B_users',
#     'Speedboat_Island_A_Island_B_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_B_Island_C_users',
#     'Speedboat_Island_B_Island_A_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_C_Island_B_users'
# ]

# ferry_price_policy(num_commuters, islands, fares, metrics_to_plot)


Capacity

In [33]:
# def vary_capacity_experiment(num_commuters, islands, capacity, metrics_to_plot, ferry_base_price):
#     '''
#     This function executes the capacity experiment. 
#     args:
#         num_commuters:   The number of agents in the model
#         islands:         The names of the islands included in the model
#         capacities:      A list or array of integers, with the lowest integer the number of agents
#         metrics_to_plot: A list of all the metrics you want to plot; 
#                          This should be a list of strings with the names of the metrics.
#                          Also, this variable might need adjustment for different number of islands
#     Output:
#         Plot of the percentage of ferry users for this settings of the experiment. 
#     '''
#     simulation = Simulation(
#         num_commuters=num_commuters,
#         num_days=60,
#         islands=islands,
#         capacity=capacity,
#         ferry_base_price=ferry_base_price,
#         ferry_base_time=40,
#         speedboat_base_price=6,
#         speedboat_base_time=10,
#     )

#     simulation.run()

#     # Plot specific results
#     metrics_to_plot = [
#         'Ferry_Island_A_Island_B_users', 'Ferry_Island_A_Island_C_users', 'Ferry_Island_B_Island_C_users',
#         'Ferry_Island_B_Island_A_users', 'Ferry_Island_C_Island_A_users', 'Ferry_Island_C_Island_B_users',
#         'Speedboat_Island_A_Island_B_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_B_Island_C_users',
#         'Speedboat_Island_B_Island_A_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_C_Island_B_users'
#     ]

#     return simulation.return_percentage_ferry_users(metrics_to_plot)

In [34]:
# islands = ["Island_A", "Island_B", "Island_C"]
# num_commuters = 1000
# prices = np.arange(1, 11, 1)
# capacities = np.arange(200, 2200, 200)
# runs = 2
# ferry_base_price = 1

# num_days = 60
# metrics_to_plot = [
#     'Ferry_Island_A_Island_B_users', 'Ferry_Island_A_Island_C_users', 'Ferry_Island_B_Island_C_users',
#     'Ferry_Island_B_Island_A_users', 'Ferry_Island_C_Island_A_users', 'Ferry_Island_C_Island_B_users',
#     'Speedboat_Island_A_Island_B_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_B_Island_C_users',
#     'Speedboat_Island_B_Island_A_users', 'Speedboat_Island_A_Island_C_users', 'Speedboat_Island_C_Island_B_users'
# ]

# percentages_ferry_users = np.zeros((len(capacities), len(prices), num_days, runs))
# mean = np.zeros((len(capacities), len(prices), num_days))
# CI = np.zeros((len(capacities), len(prices), num_days, 2))

# mean_mean = np.zeros((len(capacities), len(prices)))
# CI_mean = np.zeros((len(capacities), len(prices), 2))
# capacity = 1000

# for price_ix, ferry_base_price in enumerate(prices):
#     for capacity_ix, capacity in enumerate(capacities):
#         for i in range(runs):
#             percentages_ferry_users[capacity_ix, price_ix, :, i] = vary_capacity_experiment(num_commuters, islands, capacity, metrics_to_plot, ferry_base_price)

#         mean[capacity_ix, price_ix] = np.mean(percentages_ferry_users[capacity_ix, price_ix], axis=1)
#         CI[capacity_ix, price_ix] = np.percentile(percentages_ferry_users[capacity_ix, price_ix], [2.5, 97.5], axis=1).T

#         mean_mean[capacity_ix, price_ix] = np.mean(mean[capacity_ix, price_ix, -10:])
#         CI_mean[capacity_ix, price_ix] = np.percentile(mean[capacity_ix, price_ix, -10:], [2.5, 97.5])


In [35]:
# for price_ix, ferry_base_price in enumerate(prices):
#     plt.figure()
#     for capacity_ix, capacity in enumerate(capacities):
#         plt.plot(mean[capacity_ix, price_ix], label=capacity)
#         plt.fill_between(np.arange(0, num_days, 1), CI[capacity_ix, price_ix, :, 0], CI[capacity_ix, price_ix, :, 1], alpha=0.5)
#         plt.title(f'Percentage of ferry users in time for price = {ferry_base_price}')
#     plt.legend()
#     plt.show()

In [36]:
# for capacity_ix, capacity in enumerate(capacities):
#     plt.figure()
#     for price_ix, ferry_base_price in enumerate(prices):
#         plt.plot(mean[capacity_ix, price_ix], label=ferry_base_price)
#         plt.fill_between(np.arange(0, num_days, 1), CI[capacity_ix, price_ix, :, 0], CI[capacity_ix, price_ix, :, 1], alpha=0.5)
#         plt.title(f'Percentage of ferry users in time for capacity = {capacity}')
#     plt.legend()
#     plt.show()